In [9]:
import argparse
import collections
import re
import spacy
from tqdm import tqdm
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [10]:
def remove_tables(doc):
    line_counter = 0
    line_counter_number = 0
    table_status = False
    text = []
    possible_text = []
    for line in doc.split('\n'):

        if '-'*15 in line:
            table_status = True
            if len(possible_text) == 0:
                pass
            elif len(possible_text) > 3:
                text.extend(possible_text)
                possible_text = []
            else:
                possible_text = []
            continue
        if table_status:
            if re.search(r'\d+',line):
                if line.count(' ') < len(line.split()) + 5:
                    possible_text.append(line)
                else:
                    line_counter_number += 1
            else:
                possible_text.append(line)
            line_counter += 1
        if line_counter == 3:
            if len(possible_text) == line_counter:
                text.extend(possible_text)
                table_status = False
            else:
                possible_text = []
            line_counter = 0
            line_counter_number = 0
            continue


        if not table_status:
            text.append(line)
    return ''.join(text)

In [11]:
def load_corpus(path):
    print('Loading corpus...')
    corpus = {}
    n_text = 0
    with open(path, errors='ignore') as f:
        all_text = ''.join(f.read())
        for raw_text in tqdm(all_text.split('</DOC>')):
            if not raw_text:
                continue
            result = re.search(r'\<DOCNO\>(.*)\<\/DOCNO\>', raw_text, flags=re.MULTILINE|re.DOTALL)
            if not result:
                continue
            doc_id = result.group(1)
            doc_id = doc_id.strip()

            result = re.search(r'\<TEXT\>(.*)\<\/TEXT\>', raw_text, flags=re.MULTILINE|re.DOTALL)
            doc_text = ''
            if result:
                doc_text = result.group(1)
                doc_text = remove_tables(doc_text)
                doc_text = doc_text.replace('<P>', ' ').replace('</P>', ' ')
                doc_text = doc_text.strip()
                if doc_text:
                    n_text += 1

            corpus[doc_id] = ' '.join(doc_text.split())

    print(f'Loaded {len(corpus)} docs, {n_text} with texts.')
    return corpus

In [4]:
corpus = load_corpus('repos/pygaggle/data/robust04/trec_disks_4_and_5_concat.txt')

Loading corpus...


100%|██████████| 528160/528160 [01:11<00:00, 7345.97it/s] 

Loaded 528156 docs, 522396 with texts.


In [14]:
df = pd.read_csv('repos/pygaggle/data/robust04/run.robust04.bm25.txt', sep=' ', header=None)
bm25_docs = df.iloc[:,2].to_list()
empty_docs = [i for i in corpus.keys() if corpus[i]=='']
empty_bm25 = [i for i in empty_docs if i in bm25_docs]

In [ ]:
import re
line_counter = 3
table_status = False
text = []
lines = doc.split('\n')
for num, line in enumerate(lines):
    if '-'*15 in line:
        table_status = True
        line_counter = 3
        continue
    if table_status:
        next_lines = lines[num:num+3]
        if all([re.search(r'\d+',line) for line in lines]):
            line_counter = 3
        else:
            
        continue
    